In [1]:
import keras
import numpy as np
import pandas as pd
from time import time
#from utils import vis
from keras.models import Sequential
from keras.layers import LSTM, Dense
#from keras.utils import to_categorical
from keras.utils.np_utils import to_categorical
from keras.callbacks import TensorBoard
from sklearn.preprocessing import LabelEncoder
from tensorflow.python.client import device_lib
from sklearn.model_selection import train_test_split

In [2]:
# get data
data = pd.read_csv('TL_Level_3a_NonPath_SP1_ZyMo_PSL_1_2_3_4_546.csv') 

# split data
X=data.iloc[:,1:51]
y=data.iloc[:,151].astype('category')

print(X)
print(y)

         Xc1.A      Xc1.R     Xc1.N  ...  Xc2.lambda28  Xc2.lambda29  Xc2.lambda30
0    14.767060  10.829177  7.383530  ...      0.025877      0.025830      0.026292
1    11.728905   8.138424  3.351116  ...      0.025056      0.025025      0.026281
2     8.319776   2.773259  1.663955  ...      0.022828      0.023152      0.024468
3    10.255686   3.778411  2.428978  ...      0.023815      0.022707      0.024686
4     6.745884   2.698354  0.809506  ...      0.024331      0.024572      0.023173
..         ...        ...       ...  ...           ...           ...           ...
541   4.938105   2.222147  1.975242  ...      0.021629      0.022954      0.026503
542   4.539483   2.269741  1.765354  ...      0.024319      0.025523      0.025242
543   4.590172   4.335163  3.570134  ...      0.025397      0.025381      0.024274
544  10.080994   8.113971  3.688169  ...      0.024622      0.025884      0.026559
545   3.809671   2.913278  1.568688  ...      0.028628      0.025727      0.026259

[54

In [3]:
# init
n_epochs = 1000
n_comps = X.shape[1]

In [4]:
# encode the classification labels
#le = LabelEncoder()
#yy = to_categorical(le.fit_transform(y))

from collections import Counter
from imblearn.over_sampling import SMOTE 
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis


lda = LinearDiscriminantAnalysis(n_components=10)
X_ld = lda.fit(X, y).transform(X)

le = LabelEncoder()
yy = to_categorical(le.fit_transform(y))

sm = SMOTE(random_state=45)
x_b, y_b = sm.fit_resample(X_ld, yy)

#print('Resampled dataset shape %s' % Counter(Y_res))

# split the dataset
x_train, x_test, y_train, y_test = train_test_split(x_b, y_b, test_size=0.2,
                                                    random_state=True,
                                                    shuffle=True)

# exapand dimensions
x_train = np.expand_dims(x_train, axis=2)
x_test = np.expand_dims(x_test, axis=2)

/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/discriminant_analysis.py:463: ChangedBehaviorWarning: n_components cannot be larger than min(n_features, n_classes - 1). Using min(n_features, n_classes - 1) = min(50, 4 - 1) = 3 components.
  ChangedBehaviorWarni

In [5]:
# define model
model = Sequential()
model.add(LSTM(units=n_comps, return_sequences=True,
               input_shape=(x_train.shape[1], 1),
               dropout=0.2, recurrent_dropout=0.2))
model.add(LSTM(128, activation='tanh', return_sequences=True, dropout=0.2, recurrent_dropout=0.2))
model.add(LSTM(64, activation='tanh',  dropout=0.2, recurrent_dropout=0.2))



model.add(Dense(4 ,activation='softmax'))

# print model architecture summary
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 3, 50)             10400     
_________________________________________________________________
lstm_1 (LSTM)                (None, 3, 128)            91648     
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dense (Dense)                (None, 4)                 260       
Total params: 151,716
Trainable params: 151,716
Non-trainable params: 0
_________________________________________________________________
None


In [6]:
# compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
#model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [7]:
history = model.fit(x_train, y_train, epochs=n_epochs, batch_size=32,
                    validation_data=(x_test, y_test))

Epoch 1/1000
20/20 [==============================] - 35s 170ms/step - loss: 0.6824 - accuracy: 0.3795 - val_loss: 0.6308 - val_accuracy: 0.4051
Epoch 2/1000
20/20 [==============================] - 0s 21ms/step - loss: 0.5892 - accuracy: 0.4375 - val_loss: 0.4917 - val_accuracy: 0.4304
Epoch 3/1000
20/20 [==============================] - 0s 20ms/step - loss: 0.4750 - accuracy: 0.4074 - val_loss: 0.4666 - val_accuracy: 0.4177
Epoch 4/1000
20/20 [==============================] - 0s 21ms/step - loss: 0.4724 - accuracy: 0.4386 - val_loss: 0.4599 - val_accuracy: 0.5063
Epoch 5/1000
20/20 [==============================] - 0s 20ms/step - loss: 0.4738 - accuracy: 0.4946 - val_loss: 0.4538 - val_accuracy: 0.4810
Epoch 6/1000
20/20 [==============================] - 0s 20ms/step - loss: 0.4582 - accuracy: 0.5217 - val_loss: 0.4478 - val_accuracy: 0.5063
Epoch 7/1000
20/20 [==============================] - 0s 21ms/step - loss: 0.4572 - accuracy: 0.4756 - val_loss: 0.4479 - val_accuracy: 0.53

In [8]:
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
#from sklearn.metrics import roc_auc_score
import itertools

train_pred = model.predict(x_train)
test_pred = model.predict(x_test)
print("train-acc = " + str(accuracy_score(np.argmax(y_train, axis=1), np.argmax(train_pred, axis=1))))
print("test-acc = " + str(accuracy_score(np.argmax(y_test, axis=1), np.argmax(test_pred, axis=1))))
print("train-CFR = " + str(classification_report(np.argmax(y_train, axis=1), np.argmax(train_pred, axis=1))))
print("test-CFR = " + str(classification_report(np.argmax(y_test, axis=1), np.argmax(test_pred, axis=1))))
#print((roc_auc_score(np.argmax(y_train, axis=1), np.argmax(train_pred, axis=1))))

train-acc = 0.8904761904761904
test-acc = 0.7088607594936709
train-CFR =               precision    recall  f1-score   support

           0       0.87      0.99      0.93       151
           1       0.88      0.76      0.82       161
           2       0.98      0.90      0.94       165
           3       0.83      0.92      0.87       153

    accuracy                           0.89       630
   macro avg       0.89      0.89      0.89       630
weighted avg       0.89      0.89      0.89       630

test-CFR =               precision    recall  f1-score   support

           0       0.76      0.96      0.85        46
           1       0.68      0.58      0.63        36
           2       0.71      0.62      0.67        32
           3       0.66      0.61      0.64        44

    accuracy                           0.71       158
   macro avg       0.70      0.69      0.69       158
weighted avg       0.70      0.71      0.70       158

